# CRIO Timestamp issue

Craig Lage - 03-Sep-24

In [ ]:
import nest_asyncio
nest_asyncio.apply()
import sys, time, os, asyncio
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from lsst.daf.butler import Butler

import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.efdUtils import calcNextDay
from lsst.summit.utils.utils import dayObsIntToString
from astro_metadata_translator import ObservationInfo

from lsst_efd_client import EfdClient
from lsst_efd_client import merge_packed_time_series as mpts


In [ ]:
def _getEfdData(client, dataSeries, startTime, endTime):
    """A synchronous wrapper for geting the data from the EFD.

    This exists so that the top level functions don't all have to be async def.
    """
    loop = asyncio.get_event_loop()
    return loop.run_until_complete(client.select_time_series(dataSeries, ['*'], startTime.utc, endTime.utc))


In [ ]:
butler = Butler('/repo/embargo', collections=["LATISS/raw/all"])
client = EfdClient('usdf_efd')

In [ ]:
expId = 2025012200264
#expId = 2024090500003
#expId = 2024082800259
dataId = {'detector':0, 'exposure':expId}
expRecord = butlerUtils.getExpRecordFromDataId(butler, dataId)

tStart = expRecord.timespan.begin.tai.to_value("isot")
tEnd = expRecord.timespan.end.tai.to_value("isot")
t_start = Time(tStart, scale='tai')
t_end = Time(tEnd, scale='tai')
print(t_start, t_end)

mount_position = _getEfdData(client, "lsst.sal.ATMCS.mount_AzEl_Encoders", t_start, t_end)

az = mpts(mount_position, 'azimuthCalculatedAngle')
el = mpts(mount_position, 'elevationCalculatedAngle')
cRIO_ts = mount_position["cRIO_timestamp"]
timestamps = cRIO_ts.values
print(len(timestamps))
deltaTs = []
for n in range(1, len(timestamps)):
    deltaTs.append(timestamps[n] - timestamps[n-1])
sig = np.std(deltaTs)
%matplotlib inline
plt.title(f"cRIO timestamps deltas {expId}")
plt.plot(deltaTs, marker='x')
plt.text(2.0, 2.2,f"DeltaT sigma = {sig:.2g}")
plt.xlabel("Time(sec)")
plt.ylabel("DeltaT between successive timestamps")
plt.ylim(0, 2.5)
plt.savefig(f"/home/c/cslage/u/AuxTel/mount_graphs/cRIO_Timestamps_{expId}.png")

In [ ]:
len(mount_position)

In [ ]:
expId = 2025012100475
#expId = 2024090200356
#expId = 2024082800259
dataId = {'detector':0, 'exposure':expId}
expRecord = butlerUtils.getExpRecordFromDataId(butler, dataId)

tStart = expRecord.timespan.begin.tai.to_value("isot")
tEnd = expRecord.timespan.end.tai.to_value("isot")
t_start = Time(tStart, scale='tai')
t_end = Time(tEnd, scale='tai')
print(t_start, t_end)

mount_position = _getEfdData(client, "lsst.sal.ATMCS.measuredTorque", t_start, t_end)

az_m1 = mpts(mount_position, 'azimuthMotor1Torque')
cRIO_ts = mount_position["cRIO_timestamp"]
timestamps = cRIO_ts.values
print(len(timestamps))
deltaTs = []
for n in range(1, len(timestamps)):
    deltaTs.append(timestamps[n] - timestamps[n-1])
sig = np.std(deltaTs)
%matplotlib inline
plt.title(f"cRIO timestamps deltas {expId}")
plt.plot(deltaTs, marker='x')
plt.text(2.0, 2.2,f"DeltaT sigma = {sig:.2g}")
plt.xlabel("Time(sec)")
plt.ylabel("DeltaT between successive timestamps")
plt.ylim(0, 2.5)
plt.savefig(f"/home/c/cslage/u/AuxTel/mount_graphs/cRIO_Timestamps_Torque_{expId}.png")